# 4. VGG16

In [8]:
from keras.applications.vgg16 import VGG16 
from keras.models import Model
from keras.layers import Flatten, Dense, GlobalAveragePooling2D
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

data_path  = "/media/ubuntu16/Documents/datasets/SonarSet/SIM/Track-I-II/"
filename = "2019-06-28-"

batch_size = 128
epochs = 300

lengths = [16, 32, 48]
dim = 2
width = 192

hists = {}
errors = {}

model_path = "/home/ubuntu16/catkin_ws/src/sonar_navigation/h5/track-I-II/"

K.set_image_data_format("channels_first")

for length in lengths:
    # Loading dataset
    height = length*(width//16)
    prefix = 'vggnet-'+str(length)
    
    csv_name = data_path+filename+str(length)+'.csv'
    training_set, validation_set = csv_read(csv_name, length, dim=2, split=0.2)

    train_generator = data_generator(training_set, 
                                     batch_size,
                                     dim=2,
                                     shape=(height,width))
    valid_generator = data_generator(validation_set, 
                                     batch_size,
                                     dim=2,
                                     shape=(height,width))
    
    training_steps = len(training_set)//batch_size
    validation_steps = len(validation_set)//batch_size

    # 构建不带分类器的预训练模型
    # 因不包含top层，输入维度不用固定
    base_model = VGG16(weights='imagenet', include_top=False)
    print('VGG16 Model has been loaded.')

    # 
    base_output = base_model.output 
    x = GlobalAveragePooling2D()(base_output)
    #x = Flatten(input_shape=base_model.output_shape[1:])(x)

    # 添加全连接层
    fc1 = Dense(256, activation='relu', name='fc_1')(x)

    # 添加输出 
    steering = Dense(1, name='output')(fc1)

    # 构建完整模型
    model = Model(inputs=base_model.input, outputs=steering)

    # 首先，锁住预训练好的卷积层， 仅训练顶部的两层
    for i, layer in enumerate(base_model.layers):
        layer.trainable = False
        print(i,layer.name)
    
    # 编译模型，一定要在锁层以后操作
    optimizer = optimizers.Adam(decay=1e-5)
    model.compile(loss='mean_absolute_error', 
                  optimizer=optimizer, 
                  metrics=['mean_absolute_error'])
    
    model.summary()
    
    model_checkpoint = ModelCheckpoint(model_path+prefix+'.h5', 
                                       monitor='val_mean_absolute_error',
                                       save_best_only=True)
    model_callbacks = [model_checkpoint]

    # 在新的数据集上训练 新添加的top层
    hist_key = prefix + '-top' 
    hists[hist_key] = model.fit_generator(train_generator,
                                          samples_per_epoch=training_steps,
                                          epochs = 50,
                                          verbose = 1,
                                          validation_data = valid_generator,
                                          validation_steps= validation_steps,
                                          callbacks=model_callbacks)


    # 微调VGG16部分卷积层
    # 通过查看各层编号和名字，锁住前15层，训练之后的层
    # 不能微调所有层，因为整个网络的表示能力过于强大，很容易过拟合
    for layer in model.layers[:15]:
        layer.trainable = False

    for layer in model.layers[15:]:
        layer.trainable = True
    
    # 重新编译模型，使设置生效 （注意使用sgd）
    # compile the model with a SGD/momentum optimizer
    # and a very slow learning rate.
    model.compile(loss='mean_absolute_error',
                  optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
                  metrics=['mean_absolute_error'])

    model.summary()

    # fine-tune the model
    hists[prefix+'tune'] = model.fit_generator(train_generator,
                                               samples_per_epoch=training_steps,
                                               epochs = epochs,
                                               verbose = 1,
                                               validation_data = valid_generator,
                                               validation_steps= validation_steps, 
                                               callbacks=model_callbacks)
    
    # Evalute model
    best_model = load_model(model_path+prefix+'.h5')
    _, train_mae = best_model.evaluate_generator(train_generator,
                                                 setps=training_steps,
                                                 verbose=0)
    _, val_mae = best_model.evaluate_generator(valid_generator,
                                                 setps=validation_steps_steps,
                                                 verbose=0)
    errors[prefix+'-train_mae'] = train_mae
    errors[prefix+'-valid_mae'] = valid_mae
    
    

Starting reading data from csv file...
The length of time sequence in this file is 16
Now we 40154 data have been saved in ultrasonic type
The training set size is 32123
The validation set size is 8031


/home/ubuntu16/py2env/local/lib/python2.7/site-packages/keras/applications/vgg16.py:188: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '
/home/ubuntu16/py2env/lib/python2.7/site-packages/ipykernel_launcher.py:91: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., verbose=1, validation_data=<generator..., steps_per_epoch=250, epochs=50, callbacks=[<keras.ca..., validation_steps=62)`


VGG16 Model has been loaded.
(0, 'input_5')
(1, 'block1_conv1')
(2, 'block1_conv2')
(3, 'block1_pool')
(4, 'block2_conv1')
(5, 'block2_conv2')
(6, 'block2_pool')
(7, 'block3_conv1')
(8, 'block3_conv2')
(9, 'block3_conv3')
(10, 'block3_pool')
(11, 'block4_conv1')
(12, 'block4_conv2')
(13, 'block4_conv3')
(14, 'block4_pool')
(15, 'block5_conv1')
(16, 'block5_conv2')
(17, 'block5_conv3')
(18, 'block5_pool')
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0        

ValueError: Error when checking input: expected input_5 to have shape (3, None, None) but got array with shape (1, 192, 192)

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2,1, figsize=(10, 20))
axes[0].plot(hist_top.history['loss'], label='train')
axes[0].plot(hist_top.history['val_loss'], label='validation')

axes[1].plot(hist_tune.history['loss'], label='train')
axes[1].plot(hist_tune.history['val_loss'], label='validation')

plt.legend()
plt.show()
